In [ ]:
import sys

sys.path.append("..")

from src.auth.aurora import get_sales_client, get_portal_client

In [ ]:
sales_client = get_sales_client()
portal_client = get_portal_client()

In [ ]:
sales_crm_df = sales_client.query_to_pandas(
    """SELECT
        m.portal_client_id,
        c.client_id,
        s.device_id,
        item_code,
        s.subscription_type,
        s.start_date,
        s.expiry_date,
        c.name,
        cancel.cancel_date,
        (
                CASE WHEN s.pay_frequency = "Monthly" THEN s.unit_price
                ELSE s.unit_price / s.contract_term
                END
        ) AS mrr
    FROM sales.t_subscription AS s
    LEFT JOIN sales.t_cancellation AS cancel
    USING(subscription_id)
    LEFT JOIN sales.t_orders AS o
    USING(order_number)
    LEFT JOIN sales.t_client AS c
    USING(client_id)
    LEFT JOIN sales.t_portal_match AS m
    USING(client_id)
    WHERE item_code IN (
        '101-307-01',
        '101-309-01'
    )
    AND start_date <= curdate()
    AND (
        expiry_date IS NULL
        OR expiry_date > curdate()
    )
    AND (
        cancel_date IS NULL
        OR cancel_date > curdate()
    )
""",
)

In [ ]:
portal_clients_df = portal_client.query_to_pandas(
    """WITH RECURSIVE category_path AS (
        SELECT id, name, parent_id, 0 AS depth, id AS root_id, created_at, deleted_at
        FROM api_clients
        WHERE parent_id = 1

        UNION ALL

        SELECT c.id, c.name, c.parent_id, cp.depth + 1, cp.root_id, c.created_at, c.deleted_at
        FROM api_clients c
        INNER JOIN category_path cp ON cp.id = c.parent_id
    )
    SELECT id, name, parent_id, depth, root_id, created_at, deleted_at
    FROM category_path"""
)

In [6]:
merged_df = sales_crm_df.merge(
    portal_clients_df,
    left_on="portal_client_id",
    right_on="id",
    how="left"  # or "inner", as needed
)

In [7]:
merged_df[merged_df["root_id"] == 8124]

,portal_client_id,client_id,device_id,item_code,subscription_type,start_date,expiry_date,name_x,cancel_date,mrr,id,name_y,parent_id,depth,root_id,created_at,deleted_at
47,8124,03dd9f60-4203-11ec-be49-061393e0f1a7,Cody Park CRM,101-307-01,New Business,2024-10-01,2025-09-30,EA-RS Fire Engineering Ltd,None,150.00,8124,EA-RS Fire Engineering Ltd,1,0,8124,2020-11-25 11:57:05,NaT
48,8124,03dd9f60-4203-11ec-be49-061393e0f1a7,Spring Park CRM,101-307-01,New Business,2024-10-01,2025-09-30,EA-RS Fire Engineering Ltd,None,150.00,8124,EA-RS Fire Engineering Ltd,1,0,8124,2020-11-25 11:57:05,NaT
49,8124,03dd9f60-4203-11ec-be49-061393e0f1a7,Meridian Park CRM,101-307-01,New Business,2024-10-01,2025-09-30,EA-RS Fire Engineering Ltd,None,150.00,8124,EA-RS Fire Engineering Ltd,1,0,8124,2020-11-25 11:57:05,NaT
